In [1]:
# Import the dependencies

# Import numpy for mathematical functions and values
import numpy as np

# Import the citypy library to use a function to obtain the nearest city; note that citipy is installed in our PythonData environment
from citipy import citipy

# Import the API key that is required for OpenWeather API requests; the config.py file is a private file that is not within the GitHub repository
from config import weather_api_key

# Import the requests library for connecting with API endpoints
import requests

# Import time library for staggering the API requests (to avoid overloading the API endpoint with data requests)
import time

# Import pandas for ETL processes
import pandas as pd

# Import Matplotlib for graphing
import matplotlib.pyplot as plt

In [2]:
# Create a set of random latitude and longitude pairs using the np.random function
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Pair the latitudes and longitudes values together
lat_lngs = zip(lats, lngs)

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities information that will be displayed on a map
cities = []

# Identify the nearest city for each latitude and longitude combination that we randomly generated above
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [5]:
# Define the OpenWeather API endpoint URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Define an empty list to hold the retrieved weather data
city_data = []

# Define a record counter for keeping track of how many city records have been retrieved within a set
record_count = 1

# Define a set counter for keeping track of the set of city records that have been retrieved from the API call
# In other words, each set corresponds to a custer of API GET requests for a cluster of cities
set_count = 1

# Print the beginning of the logging process
print("-----------------------------")
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in the cities list using the built-in enumerate function
for i, city in enumerate(cities):

    # Group cities in sets of 50 to avoid overloading the OpenWeather API endpoint with GET requests
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an API endpoint URL with a city embedded within it
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the current record count, set count, and city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Increment the record counter
    record_count += 1
    
    # Run an API GET request for each of the cities in the cities array
    try:
        # Retrieve and parse the JSON data from the OpenWeather API GET request
        city_weather = requests.get(city_url).json()

        # Extract the desired data from the retrieved weather JSON data
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]

        # Append a city's weather information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# In case there is an error in retrieving a city's weather data, skip that city
    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | puerto ayora
Processing Record 2 of Set 1 | kamaishi
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | qaqortoq
Processing Record 5 of Set 1 | cape town
Processing Record 6 of Set 1 | pundaguitan
Processing Record 7 of Set 1 | kodiak
Processing Record 8 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 9 of Set 1 | vaini
Processing Record 10 of Set 1 | zig
Processing Record 11 of Set 1 | ahipara
Processing Record 12 of Set 1 | brae
Processing Record 13 of Set 1 | new norfolk
Processing Record 14 of Set 1 | hobart
Processing Record 15 of Set 1 | albany
Processing Record 16 of Set 1 | westport
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | tamandare
Processing Record 19 of Set 1 | tuktoyaktuk
Processing Record 20 of Set 1 | rio grande
Processing Record 21 of Set 1 | ulychne
Processing Record 22 of S

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

# Check to see if the data was properly loaded
city_data_df.head(10)

In [ ]:
# Create and output a .csv file

# Define the output file name and its directory
output_data_file = "weather_database/weatherpy_database.csv"

# Export the city_data_df to the the previously defined .csv file
city_data_df.to_csv(output_data_file, index_label = "City_ID")